<a href="https://cognitiveclass.ai/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDL0101ENSkillsNetwork20718188-2022-01-01"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Regression Models with Keras</font></h1>


<h1>Graded Assignment: Building Regression Model with Keras</h1>



<a id="item31"></a>


# A. Building a baseline Keras Regression model 

### Task:

A. Build a baseline model (5 marks) 

Use the Keras library to build a neural network with the following:

- One hidden layer of 10 nodes, and a ReLU activation function

- Use the adam optimizer and the mean squared error  as the loss function.

1. Randomly split the data into a training and test sets by holding 30% of the data for testing. You can use the train_test_splithelper function from Scikit-learn.

2. Train the model on the training data using 50 epochs.

3. Evaluate the model on the test data and compute the mean squared error between the predicted concrete strength and the actual concrete strength. You can use the mean_squared_error function from Scikit-learn.

4. Repeat steps 1 - 3, 50 times, i.e., create a list of 50 mean squared errors.

5. Report the mean and the standard deviation of the mean squared errors.

Submit your Jupyter Notebook with your code and comments.

## Importing the libraries.


In [1]:
import pandas as pd
import numpy as np

## Download and Clean Dataset


In [2]:
concrete_df = pd.read_csv('concrete_data.csv')
concrete_df.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


So the first concrete sample has 540 cubic meter of cement, 0 cubic meter of blast furnace slag, 0 cubic meter of fly ash, 162 cubic meter of water, 2.5 cubic meter of superplaticizer, 1040 cubic meter of coarse aggregate, 676 cubic meter of fine aggregate. Such a concrete mix which is 28 days old, has a compressive strength of 79.99 MPa.


#### Let's check how many data points we have.


In [4]:
concrete_df.shape

(1030, 9)

The dataset has 1030 samples (instances) with features. to train our model on. 
Data size is small so caution to train the model to prevent overfitting during training 

Let's check the dataset for any missing values.


In [5]:
concrete_df.describe() # to reveal quick statistics about the data

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [6]:
concrete_df.isnull().sum() # checking if there are missing data but found none. Data clean enough to be processed

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

The data looks very clean and is ready to be used to build our model.


## Data preprocessing 
### Split data into predictors and target


##### The target (y) in this problem is the concrete sample strength which is a continous variable and the predictors (X) are the other columns.

In [8]:
X = concrete_df.drop('Strength',axis=1)  # Take all columns except Strength
y = concrete_df['Strength']  # Take only Strength column

<a id="item2"></a>


Let's do a quick sanity check of the predictors and the target dataframes.


In [9]:
X.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [10]:
y.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

<a id='item34'></a>

### Train|Test Split of Data 

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42) # 30% of data kept to test the model

<a id='item34'></a>

Let's save the number of predictors to *n_cols* since we will need this number when building our network.


In [13]:
n_cols = X_train.shape[1] # number of predictors entering the Deep Learning model 
n_cols

8

<a id='item32'></a>


## Baseline Keras Neural Network model building


In [14]:
# Import useful model buiding libraries

import keras
from keras.models import Sequential
from keras.layers import Dense

### Creating a function that defines the regression model 

<a id="item4"></a>


In [15]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(10, activation='relu')) # one hidden layer
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mse'])
    return model

<a id='item34'></a>


## Model Training  and Testing 


Train and test of the model at the same time using the *fit* method. Note 30% of the data (i.e test_set) is used to evaluate the model during trainig with 50 epochs.


In [16]:
# build the regression model by calling the prepared function

model = regression_model()
# fit the model
score = model.fit(X_train, y_train,  validation_data=(X_test,y_test), epochs=50, verbose=0)

# printing list of 50 test mean square error (Test_MSE): obtained by passing X_test and y_test as validation data
Test_MSE = score.history['val_mse'] 
print('List of Test_MSE:')
print(" ")
print(np.round(Test_MSE, 2))
print(" ")

# Reporting the mean and the standard deviation of the test mean squared errors.
#print()
print('The mean and standard deviation of the mean squared errors are:', "",
      "%.2f%% (+/- %.2f%%)" % (np.mean(Test_MSE),np.round(np.std(Test_MSE), 2)))

List of Test_MSE:
 
[1707.64  816.84  496.99  327.21  273.29  248.75  224.66  211.86  206.25
  189.38  186.83  174.32  168.18  176.46  160.47  156.71  153.86  150.1
  152.66  146.54  151.49  141.54  140.18  140.76  136.4   139.    130.63
  130.84  126.38  124.59  125.93  123.73  121.06  120.7   118.94  123.22
  125.55  120.83  114.88  114.44  114.13  115.88  115.76  113.2   116.13
  111.18  110.43  108.99  112.1   110.49]
 
The mean and standard deviation of the mean squared errors are:  200.57% (+/- 243.40%)


<a id='item34'></a>

<a id='item34'></a>

# B. Building a baseline Keras Regression model 

### Task continued from part A:
B. Normalize the data (5 marks) 

Repeat Part A but use a normalized version of the data. Recall that one way to normalize the data is by subtracting the mean from the individual predictors and dividing by the standard deviation.

How does the mean of the mean squared errors compare to that from Step A?

<a id='item34'></a>

## Scaling (Normalization)

In [18]:
scaler = StandardScaler()
scaled_X_train = scaler.fit_transform(X_train)  
scaled_X_test = scaler.transform(X_test)      # Not calling fit on X_test to prevent data leakage

In [19]:
scaled_X_train

array([[-0.8284837 , -0.85529604,  0.76170123, ...,  0.41554504,
         1.67680283, -0.29298042],
       [ 0.37482257, -0.85529604, -0.81691314, ...,  1.13697904,
         0.14190356, -0.63384539],
       [ 0.31756625,  1.56893487, -0.81691314, ..., -1.55119111,
         1.35833309, -0.69877204],
       ...,
       [-0.86911722, -0.85529604,  1.1496791 , ...,  1.34091798,
         0.34047881,  0.87569944],
       [ 1.76005619,  0.49999164, -0.81691314, ..., -1.55119111,
         0.12317004, -0.29298042],
       [ 0.27323877, -0.85529604,  0.95804631, ..., -0.62199432,
         0.14190356, -0.29298042]])

In [20]:
n_cols = scaled_X_train.shape[1] # number of predictors entering the Deep Learning model 
n_cols

8

<a id='item34'></a>

## Keras Neural Network model building with Normalized Data

In [21]:
# Import useful model buiding libraries

import keras
from keras.models import Sequential
from keras.layers import Dense

### Creating a function that defines the regression model 

In [22]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mse'])
    return model

## Model Training  and Testing with normalized data

Train and test of the model at the same time using the *fit* method. Note 30% of the data (i.e test_set) is used to evaluate the model during trainig with 50 epochs.


In [23]:
# build the regression model by calling the prepared function

model = regression_model()
# fit the model
score = model.fit(scaled_X_train, y_train,  validation_data=(scaled_X_test,y_test), epochs=50, verbose=0)

# printing list of 50 test mean square error (Test_MSE): obtained by passing scaled_X_test and y_test as validation data
Test_MSE = score.history['val_mse'] 
print('List of Test_MSE:')
print(" ")
print(np.round(Test_MSE, 2))
print(" ")

# Reporting the mean and the standard deviation of the test mean squared errors.

print('The mean and standard deviation of the mean squared errors are:', "",
      "%.2f%% (+/- %.2f%%)" % (np.mean(Test_MSE),np.round(np.std(Test_MSE), 2)))


List of Test_MSE:
 
[1475.13 1453.84 1430.84 1402.19 1364.45 1316.42 1255.9  1182.78 1098.27
  998.5   889.73  771.05  650.6   540.75  448.75  374.18  319.88  279.72
  253.59  235.88  224.61  215.82  209.72  203.61  199.24  194.82  191.23
  187.65  184.71  182.    179.18  177.11  175.14  173.45  171.94  170.37
  168.51  167.27  165.48  164.4   163.03  161.99  160.7   159.91  159.02
  157.81  156.95  156.18  154.96  154.16]
 
The mean and standard deviation of the mean squared errors are:  460.67% (+/- 450.65%)


#### Comment: The mean squared error increases from 200.57% (+/- 243.40%) in base line model  in part (A) to 460.67% (+/- 450.65%) when normalized data were used in part B. 

<a id='item34'></a>

# C. Building Keras Regression model with more epochs

### Task continued from part B:
C. Increate the number of epochs (5 marks)

Repeat Part B but use 100 epochs this time for training.

How does the mean of the mean squared errors compare to that from Step B?



## Scaling (Normalization)

In [26]:
scaler = StandardScaler()
scaled_X_train = scaler.fit_transform(X_train)  
scaled_X_test = scaler.transform(X_test)      # Not calling fit on X_test to prevent data leakage

In [27]:
n_cols = scaled_X_train.shape[1] # number of predictors entering the Deep Learning model 
n_cols

8

## Baseline Keras Neural Network model building with Normalized Data

In [28]:
# Import useful model buiding libraries

import keras
from keras.models import Sequential
from keras.layers import Dense

### Creating a function that defines the regression model 

In [29]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mse'])
    return model

## Model Training  and Testing with normalized data

Train and test of the model at the same time using the *fit* method. Note 30% of the data (i.e test_set) is used to evaluate the model during trainig with 50 epochs.


In [30]:
# build the regression model by calling the prepared function

model = regression_model()
# fit the model
score = model.fit(scaled_X_train, y_train,  validation_data=(scaled_X_test,y_test), epochs=100, verbose=0)

# printing list of 50 test mean square error (Test_MSE): obtained by passing scaled_X_test and y_test as validation data
Test_MSE = score.history['val_mse'] 
print('List of Test_MSE:')
print(" ")
print(np.round(Test_MSE, 2))
print(" ")

# Reporting the mean and the standard deviation of the test mean squared errors.

print('The mean and standard deviation of the mean squared errors are:', "",
      "%.2f%% (+/- %.2f%%)" % (np.mean(Test_MSE),np.round(np.std(Test_MSE), 2)))


List of Test_MSE:
 
[1422.22 1381.39 1334.33 1278.96 1215.04 1141.48 1057.16  963.07  861.03
  756.59  653.69  554.56  467.01  398.09  347.92  311.03  286.65  269.68
  258.09  248.95  241.06  234.17  227.75  221.98  216.52  211.03  206.13
  202.07  198.22  195.14  191.68  188.36  185.91  183.06  180.7   178.31
  176.35  174.51  172.69  170.39  168.83  167.39  165.78  164.41  162.69
  161.32  159.67  157.99  156.95  155.28  154.03  152.72  151.78  150.48
  149.7   148.3   147.29  146.68  145.4   144.19  143.33  142.39  141.62
  140.6   139.29  138.84  138.    137.08  136.38  135.74  134.99  134.09
  133.3   132.4   131.7   130.74  130.41  129.41  128.15  127.61  126.8
  126.14  125.1   124.33  123.67  123.07  121.99  121.22  120.58  119.35
  118.83  118.06  117.12  116.72  116.04  114.92  114.15  113.56  112.6
  111.7 ]
 
The mean and standard deviation of the mean squared errors are:  273.66% (+/- 308.82%)


#### Comment: The mean squared error decreases from  460.67% (+/- 450.65%) with 50 epochs in part B, to 273.66% (+/- 308.82%) in part (C) when epochs was increased to 100. This indicates the model learn and performs better with mores epochs (training cycles).

<a id='item34'></a>

# D. Building Keras Regression model with more epoch than baseline model

### Task continued from part B:
D. Increase the number of hidden layers (5 marks)

Repeat part B but use a neural network with the following instead:

- Three hidden layers, each of 10 nodes and ReLU activation function.

How does the mean of the mean squared errors compare to that from Step B?



## Scaling (Normalization)

In [31]:
scaler = StandardScaler()
scaled_X_train = scaler.fit_transform(X_train)  
scaled_X_test = scaler.transform(X_test)      # Not calling fit on X_test to prevent data leakage

In [32]:
n_cols = scaled_X_train.shape[1] # number of predictors entering the Deep Learning model 
n_cols

8

## Baseline Keras Neural Network model building with Normalized Data

In [33]:
# Import useful model buiding libraries

import keras
from keras.models import Sequential
from keras.layers import Dense

### Creating a function that defines the regression model 

In [34]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(10, activation='relu')) # 3 hidden layers
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mse'])
    return model

## Model Training  and Testing with normalized data

Train and test of the model at the same time using the *fit* method. Note 30% of the data (i.e test_set) is used to evaluate the model during trainig with 50 epochs.


In [35]:
# build the regression model by calling the prepared function

model = regression_model()
# fit the model
score = model.fit(scaled_X_train, y_train,  validation_data=(scaled_X_test,y_test), epochs=50, verbose=0)

# printing list of 50 test mean square error (Test_MSE): obtained by passing scaled_X_test and y_test as validation data
Test_MSE = score.history['val_mse'] 
print('List of Test_MSE:')
print(" ")
print(np.round(Test_MSE, 2))
print(" ")

# Reporting the mean and the standard deviation of the test mean squared errors.

print('The mean and standard deviation of the mean squared errors are:', "",
      "%.2f%% (+/- %.2f%%)" % (np.mean(Test_MSE),np.round(np.std(Test_MSE), 2)))

List of Test_MSE:
 
[1476.31 1440.55 1349.4  1177.94  905.36  571.73  347.97  274.44  237.23
  214.83  199.64  186.94  177.16  169.27  162.63  157.02  152.12  149.02
  144.77  141.8   138.91  134.92  131.23  128.21  124.92  120.7   118.23
  114.24  110.48  107.15  104.33  100.85   98.39   95.1    91.94   90.07
   86.43   84.17   80.76   79.7    74.87   75.25   70.4    69.74   66.88
   65.     62.98   60.88   59.95   58.15]
 
The mean and standard deviation of the mean squared errors are:  248.82% (+/- 356.81%)


#### Comment: The mean squared error decreases from  460.67% (+/- 450.65%) with one hidden layer in part B, 248.82% (+/- 356.81%) in part (D) when 3 hidden layers were used. This indicates the model learn and performs better with mores layer (i.e neuron).

<a id='item34'></a>

<a id='item34'></a>

This notebook is part of a course on **Coursera** called *Introduction to Deep Learning & Neural Networks with Keras*. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://cocl.us/DL0101EN_Coursera_Week3\_LAB1).


<hr>

Copyright © 2019 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_medium=dswb&utm_source=bducopyrightlink&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDL0101ENSkillsNetwork20718188-2022-01-01&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDL0101ENSkillsNetwork20718188-2022-01-01).
